In [30]:
import pandas as pd
import re

caminho_arquivo = 'junho25.xlsx'

# Carrega o conjunto de dados a partir de um arquivo Excel (.xlsx)
df = pd.read_excel(caminho_arquivo)

# Encontra os índices das linhas dos cabeçalhos 'Receitas' e 'Despesas'
# Usa correspondência exata após remover espaços em branco
linhas_cabecalho_receitas = df[df.iloc[:, 0].astype(str).str.strip() == 'Receitas']
linhas_cabecalho_despesas = df[df.iloc[:, 0].astype(str).str.strip() == 'Despesas']

indice_linha_cabecalho_receitas = -1
if not linhas_cabecalho_receitas.empty:
    indice_linha_cabecalho_receitas = linhas_cabecalho_receitas.index[0]

indice_linha_cabecalho_despesas = -1
if not linhas_cabecalho_despesas.empty:
    # Obtém o índice de 'Despesas' que vem depois de 'Receitas'
    linhas_cabecalho_despesas_apos_receitas = linhas_cabecalho_despesas[linhas_cabecalho_despesas.index > indice_linha_cabecalho_receitas]
    if not linhas_cabecalho_despesas_apos_receitas.empty:
        indice_linha_cabecalho_despesas = linhas_cabecalho_despesas_apos_receitas.index[0]


# Imprime os índices encontrados para depuração (opcional)
print(f"Índice da linha do cabeçalho de Receitas: {indice_linha_cabecalho_receitas}")
print(f"Índice da linha do cabeçalho de Despesas: {indice_linha_cabecalho_despesas}")

# Sai se os cabeçalhos não forem encontrados
if indice_linha_cabecalho_receitas == -1 or indice_linha_cabecalho_despesas == -1:
    print("Não foi possível encontrar os cabeçalhos 'Receitas' ou 'Despesas' com correspondência exata. Verifique o conteúdo do arquivo ou tente ajustar a string de busca.")
    # Exit is commented out so the script can run in the environment
    # exit() # Sai do script ou levanta um erro


# --- Processa Receitas ---
# Recorta os dados brutos para Receitas, começando da linha APÓS o cabeçalho até a linha do cabeçalho de Despesas
df_dados_brutos_receitas = df.iloc[indice_linha_cabecalho_receitas + 1 : indice_linha_cabecalho_despesas].copy()

# Atribui nomes de coluna consistentes com base nas suas posições nos dados brutos
df_receitas = pd.DataFrame()
df_receitas['Item'] = df_dados_brutos_receitas.iloc[:, 0]
df_receitas['Competência'] = df_dados_brutos_receitas.iloc[:, 1]
df_receitas['Liquidação'] = df_dados_brutos_receitas.iloc[:, 2]
df_receitas['Valor'] = df_dados_brutos_receitas.iloc[:, 4] # Coluna de Valor para Receitas (original Unnamed:4)
df_receitas['Grupo_Checker'] = df_dados_brutos_receitas.iloc[:, 5] # Coluna G para classificação (original Unnamed:5)

# Adiciona a coluna 'Tipo'
df_receitas['Tipo'] = 'Receita'

# Aplica a lógica de classificação anterior para Receitas
# Remove linhas que começam com "Total" na coluna 'Item'
df_receitas = df_receitas[~df_receitas['Item'].astype(str).str.startswith('Total', na=False)].copy()

# Cria a coluna 'Grupo' e a preenche com base na lógica revisada para Receitas
df_receitas['Grupo'] = ''
grupo_atual_receita = ''

# Itera pelas linhas usando seus índices originais para identificar grupos corretamente
for idx in df_receitas.index:
    # Verifica se esta linha é um cabeçalho de grupo (Grupo_Checker é NaN E Valor é NaN)
    eh_cabecalho_grupo = (pd.isna(df_receitas.loc[idx, 'Grupo_Checker']) or str(df_receitas.loc[idx, 'Grupo_Checker']).strip() == '') and \
                         (pd.isna(df_receitas.loc[idx, 'Valor']) or str(df_receitas.loc[idx, 'Valor']).strip() == '')

    if eh_cabecalho_grupo:
        grupo_atual_receita = df_receitas.loc[idx, 'Item']
    else:
        # Esta é uma linha de dados, atribui o grupo atual
        df_receitas.loc[idx, 'Grupo'] = grupo_atual_receita

# Para Receitas, remove linhas que são cabeçalhos de classificação e não têm um 'Valor'
df_receitas_processadas = df_receitas[
    ~((pd.isna(df_receitas['Grupo_Checker']) | (df_receitas['Grupo_Checker'].astype(str).str.strip() == '')) &
      (pd.isna(df_receitas['Valor']) | (df_receitas['Valor'].astype(str).str.strip() == '')))].copy()



# --- Processa Despesas ---
# Recorta os dados brutos para Despesas, começando da linha APÓS o cabeçalho até o final do DataFrame
df_dados_brutos_despesas = df.iloc[indice_linha_cabecalho_despesas + 1:].copy()

# Atribui nomes de coluna consistentes com base nas suas posições nos dados brutos
df_despesas = pd.DataFrame()
df_despesas['Item'] = df_dados_brutos_despesas.iloc[:, 0]
df_despesas['Competência'] = df_dados_brutos_despesas.iloc[:, 1]
df_despesas['Liquidação'] = df_dados_brutos_despesas.iloc[:, 2]
df_despesas['Documento'] = df_dados_brutos_despesas.iloc[:, 3]
df_despesas['Forma de Pgto.'] = df_dados_brutos_despesas.iloc[:, 4]
df_despesas['Grupo_Checker'] = df_dados_brutos_despesas.iloc[:, 5] # Coluna G para classificação
df_despesas['Valor'] = df_dados_brutos_despesas.iloc[:, 6] # Coluna de Valor para Despesas

# Adiciona a coluna 'Tipo'
df_despesas['Tipo'] = 'Despesa'

# Aplica a lógica de classificação anterior para Despesas
# Remove linhas que começam com "Total" na coluna 'Item'
df_despesas = df_despesas[~df_despesas['Item'].astype(str).str.startswith('Total', na=False)].copy()

# Cria a coluna 'Grupo'
df_despesas['Grupo'] = ''
grupo_atual_despesa = ''
for i in range(len(df_despesas)):
    # Verifica se 'Grupo_Checker' é NaN ou string vazia
    if pd.isna(df_despesas.iloc[i]['Grupo_Checker']) or str(df_despesas.iloc[i]['Grupo_Checker']).strip() == '':
        grupo_atual_despesa = df_despesas.iloc[i]['Item']
    else:
        df_despesas.loc[df_despesas.index[i], 'Grupo'] = grupo_atual_despesa

# Remove as linhas de classificação (onde 'Grupo_Checker' é NaN/vazio)
df_despesas_processadas = df_despesas.dropna(subset=['Grupo_Checker']).copy()



# --- Padroniza e Combina DataFrames ---
# Define as colunas finais desejadas explicitamente
colunas_finais = ['Tipo', 'Grupo', 'Item', 'Competência', 'Liquidação', 'Documento', 'Forma de Pgto.', 'Valor']

# Garante que as colunas 'Documento' e 'Forma de Pgto.' existam em receitas antes de reindexar
for col in ['Documento', 'Forma de Pgto.']:
    if col not in df_receitas_processadas.columns:
        df_receitas_processadas[col] = None
    df_receitas_processadas[col] = df_receitas_processadas[col].astype(object)

# Padroniza as colunas para df_receitas_processadas_final reindexando
df_receitas_processadas_final = df_receitas_processadas.reindex(columns=colunas_finais)

# Padroniza as colunas para df_despesas_processadas_final reindexando
df_despesas_processadas_final = df_despesas_processadas.reindex(columns=colunas_finais)

# Concatena os DataFrames processados
df_final = pd.concat([df_receitas_processadas_final, df_despesas_processadas_final], ignore_index=True)

# Converte a coluna 'Valor' para numérico, tratando vírgulas e garantindo que seja numérico
# Função para limpar e converter valores
def clean_and_convert_value(value):
    value_str = str(value).strip()
    is_negative = False

    if value_str.startswith('(') and value_str.endswith(')'):
        is_negative = True
        value_str = value_str[1:-1] # Remove parentheses

    value_str = value_str.replace('%', '') # Remove percentage sign
    value_str = value_str.replace('.', '') # Remove thousands separator (dot)
    value_str = value_str.replace(',', '.') # Replace comma with dot for decimal

    try:
        numeric_value = float(value_str)
        if is_negative:
            numeric_value *= -1
        return numeric_value
    except ValueError:
        return pd.NA # Return Not Available for values that cannot be converted

df_final['Valor'] = df_final['Valor'].apply(clean_and_convert_value)
df_final['Valor'] = pd.to_numeric(df_final['Valor'], errors='coerce')

# Substitui valores em branco ou NaN na coluna 'Forma de Pgto.' por 'Outros'
df_final['Forma de Pgto.'] = df_final['Forma de Pgto.'].replace('', pd.NA)
df_final['Forma de Pgto.'] = df_final['Forma de Pgto.'].fillna('Outros')

# Exibe as primeiras linhas e informações do DataFrame final
print("\nCabeçalho do DataFrame Final:")
print(df_final.head())

print("\nInformações do DataFrame Final:")
print(df_final.info())

# Captura a competência mais frequente na coluna "Competência"
competencia_mais_frequente = df_final['Competência'].mode(dropna=True)
if not competencia_mais_frequente.empty:
  competencia_str = competencia_mais_frequente.iloc[0].replace('/', '_')
  nome_arquivo = f'receitas_despesas_{competencia_str}.xlsx'
else:
  nome_arquivo = 'receitas_despesas.xlsx'

# Salva o DataFrame final em um novo arquivo Excel com o nome dinâmico
df_final.to_excel(nome_arquivo, index=False)

Índice da linha do cabeçalho de Receitas: 10
Índice da linha do cabeçalho de Despesas: 72

Cabeçalho do DataFrame Final:
      Tipo           Grupo                                               Item  \
0  Receita  Taxa Ordinária                                         COND. PAGO   
1  Receita  Taxa Ordinária  0501 , 1202 , 1502 MAJORAÇÃO DE TAXA APROVADA ...   
2  Receita  Taxa Ordinária  0301 , 0402 , 0601 , 0701 , 0802 , 0901 , 1101...   
3  Receita  Taxa Ordinária  0201 , 1002 , 1501 , 1602 MAJORAÇÃO DE TAXA AP...   
4  Receita  Taxa Ordinária  0101 , 0801 , 1001 , 1401 MAJORAÇÃO DE TAXA AP...   

  Competência  Liquidação Documento Forma de Pgto.    Valor  
0     06/2025  25/06/2025      None         Outros   1910.0  
1     06/2025  30/05/2025      None         Outros   5730.0  
2     06/2025  02/06/2025      None         Outros  15280.0  
3     06/2025  03/06/2025      None         Outros   7640.0  
4     06/2025  04/06/2025      None         Outros   7640.0  

Informações do Data